# Equations

## Necessary imports

In [1]:
import math
import random

## Constants

In [17]:
#Data from Fukuda and Sakagami (1968)
LIFESPANlarva = 5
MORTALITYnursing = 0.005
MORTALITYprocessing = 0.005
MORTALITYforaging = 0.035
LIFESPANpupa = 12

## 2.1.   Modeling the queen’s egg laying behavior

In [3]:
def season(t):
    # (1)
    return max(1-(1/(1+x1*math.exp(-2*t/x2))),1/(1+x3*math.exp(-2*(t-x4)/x5)))

In [4]:
def CELLSempty(t):
    # (2)
    return CELLShive - CELLSbrood(t) - STORESpollen(t) - STORESnectar(t) - STOREShoney(t)

In [5]:
def SUPcomb(t):
    # (3)
    if CELLSempty(t)/(CELLShive + 1) < SUPthreshold:
        return CELLSempty(t)/((CELLShive + 1) * SUPthreshold)
    else:
        return 1

In [6]:
def ELRstoch(t):
    # (4)
    return random.uniform(-ELRstochrange,+ELRstochrange)

In [7]:
def ELR(t):
    # (5)
    return ELRbase * (1 + ELRstoch(t)) * (1 - season(t)) * SUPcomb(t) 

## 2.2. Modeling the immature stages

### 2.2.1 Eggs

In [8]:
def EGGS(i,t):
    # (6)
    if i == 1:
        return ELR(t-1) * (1 - MORTALITYeggs)
    elif i in [2,3]:
        return EGGS(i-1,t-1) * (1 - MORTALITYeggs) 

In [9]:
def CELLSeggs(t):
    # (7)
    sum = 0
    for i in range(LIFESPANegg+1):
        if i != 0:
            sum += EGGS((i,t))
    return sum

In [10]:
def CELLSeggs(t):
    # (8)
    # alternative to (7)
    sum = 0
    for i in range(LIFESPANegg+1):
        if i != 0:
            sum += (ELR(t-i)*math.pow(1-MORTALITYeggs,i))
    return sum

### 2.2.2. Larvae (unsealed)

In [11]:
def SURVIVALlarvae(i,t):
    # (9)
    return (1 - CANNIBALISMlarvae(i,t)) * (1 - MORTALITYlarvae)

In [12]:
def CANNIBALISMlarvae(i,t):
    # (10)
    return CANNIBALISMhungerbase(i) * (1 - (INDEXpollensituation(t-1) * INDEXnursingquality(t-1)))

In [13]:
def LARVAE(i,t):
    # (11)
    if i == 1:
        return EGGS(LIFESPANegg,t-1)*SURVIVALlarvae(1,t)
    elif 1 < i and i <= LIFESPANlarva:
        return LARVAE(i-1,t-1) * SURVIVALlarvae(i,t)

In [14]:
def CELLSlarvae(t):
    # (12)
    sum = 0
    for i in range(LIFESPANlarva+1):
        if i != 0:
            sum += LARVAE((i,t))
    return sum 

In [15]:
def CELLSlarvae(t):
    # (13)
    # alternative to (12)
    
    def product(i):
        prdct = 1
        for k in range(i+1):
            if k != 0:
                prdct *= SURVIVALlarvae(k,t-i+k)
        return prdct
    
    sum = 0
    for i in range(LIFESPANlarva+1):
        if i != 0:
            sum += (ELR(t-i-LIFESPANegg)*math.pow(1-MORTALITYeggs,i)*product(i))  
    
    return sum

In [16]:
def PUPAE(i,t):
    # (14)
    if i == 1:
        return LARVAE(LIFESPANlarva,t-1) * (1 - MORTALITYpupae)
    elif 1 < i and i <= LIFESPANpupa:
        return PUPAE(i-1,t-1) * (1 - MORTALITYpupae)

In [17]:
def CELLSpupae(t):
    # (15)
    sum = 0
    for i in range(LIFESPANpupa+1):
        if i != 0:
            sum += PUPAE((i,t))
    return sum 

In [18]:
def CELLSpupae(t):
    # (16)
    # alternative to (15)
    
    def product(i):
        prdct = 1
        for k in range(LIFESPANlarva+1):
            if k != 0:
                prdct *= SURVIVALlarvae(k,t-i-LIFESPANlarva+k)
        return prdct
    
    sum = 0
    for i in range(LIFESPANpupa+1):
        if i != 0:
            sum += (ELR(t-i-LIFESPANegg-LIFESPANlarva)*math.pow(1-MORTALITYeggs,LIFESPANegg)*product(i)*math.pow(1-MORTALITYpupae,i))  
    
    return sum

In [19]:
def CELLSbrood(t):
    # (17)
    return CELLSeggs(t) + CELLSlarvae(t) + CELLSpupae(t)

## 2.3 Modeling the population of adult bees

In [9]:
def MORTALITYadult(t) :
    # (18)
	res = MORTALITYadultbase
	res += (MORTALITYnursing * NURSES(t - 1) / BEESadult(t - 1) + 1)
	res += (MORTALITYprocessing * PROCESSORS(t - 1) / BEESadult(t - 1) + 1)
	res += (MORTALITYforaging * FORAGERSactive(t - 1) / BEESadult(t - 1) + 1)
	return res

In [10]:
def BEESadult(t) : 
    # (19)
	res = (BEESadult(t - 1) + PUPAE(LIFESPANpupa, t - 1)) * (1 - MORTALITYadult(t))
	return res

## 2.4 Modeling the influence of the environment

In [11]:
def INDEXrain(t) :
    # (20)
	return (1 - RAIN(t))

In [12]:
def RAIN(t) :
    # (21)
	res = HOURSraining_during_daylight(t) / HOURSdaylight(t)
	return res

In [13]:
def INDEXtemperature(t) :
    # (22)
	if ((TEMP(t) <= 14) or (TEMP(t) > 40)) :
		res = 0
	elif (TEMP(t) <= 22) :
		res = (TEMP(t) - 14) / 8
	elif (TEMP(t) <= 32) : 
		res = 1
	else : 
		res = (40 - TEMP(t)) / 8 

	return res

In [14]:
def INDEXflight(t) : 
    # (23)
	return (INDEXrain(t) * INDEXtemperature(t))

In [15]:
def INDEXnectaroutside(t) :
    # (24)
	return min((1 - season(t)) * 1.5, 1)

In [16]:
def INDEXpollenoutside(t) :
    # (25)
	return min((1 - season(t)) * 1.5, 1)

## 2.5 Modeling task decisions

In [21]:
def NEEDworkers(t):
    # (26)
    return NEEDnurses(t) + NEEDpollenforagers(t)

In [22]:
def RATIOworkforce(t, FACTORothertasks = 0.2):
    # (27)
    return min(BEESadult(t) * (1 - FACTORothertasks) / (NEEDWORKERS(t) + 1), 1)

## 2.6 Modeling the regulation of nursing

In [23]:
def NEEDnurses(t):
    # (28)
    res = 0
    for i in range(1, LIFESPANlarva):
        res = LARVAE(i, t) * NEEDnurses_per_larva(i)
    return res + CELLSeggs(t) * NEEDnursesperegg + CELLSpupae(t) * NEEDnursesperpupa

In [24]:
def NURSES(t):
    # (29)
    return NEEDworkers(t) + RATIOworkforce(t)

In [25]:
def INDEXnursingquality(t):
    # (30)
    return NURSES(t)/(NEEDnurses(t) + 1)

## 2.7 Modeling the regulation of foraging

In [ ]:
def FORAGERS(t):
    # (31)
    res = FORAGERSpollen(t)+FORAGERSnectaractive(t)
    return(res)

In [ ]:
def FORAGERSactive(t):
    # (32)
    res = FORAGERSpollenactive(t)+FORAGERSnectaractive(t)
    return(res)

### 2.7.1 Recruitment of pollen foragers

In [ ]:
def NEEDpollen(t):
    # (33)
    res = NEEDpollen_larvae(t)+NEEDpollen_adult(t)
    return(res)

In [ ]:
def NEEDpollen_larvae(t):
    # (34)
    result = 0
    for i in range(1,LIFESPANlarva):
        S+= (PELLENNEEDlarva(i)*(LARVAE(i,t)))
    return(result)

In [ ]:
def NEEDpollen_adult(t):
    # (35)
    result = BEESadult(t) * POLLENNEEDadult + NURSES(t) * POLLENNEEDnurse
    return(result)

In [ ]:
def NEEDpollenincome(t):
    # (36)
    S = 0
    for i in range(0,2):
        S += NEEDpollen(t - d)
    tmp = S/3 * FACTORpollenstorage - STORESpollen(t)
    result = max(0,tmp)
    return(result)

In [ ]:
def NEEDpollenforagers(t):
    # (37)
    result = NEEDpollenincome(t-1) / (LOADpollenforager * TURNSpollenforager * FACTORforagingsuccess)
    return(result)

In [ ]:
def FORAGERSpollen(t):
    # (38)
    max1 = NEEDpollenforagers(t) * RATIOworkforce(t)
    max2 = (BEESadult(t)-NURSES(t))*FACTORminpollenforagers
    min1 = max(max1,max2)
    min2 = BEESadult(t) * FACTORforagingmax
    result = min(min1,min2)
    return(result)

In [ ]:
def FORAGERSpollenactive(t):
    # (39)
    result = FORAGERpollen(t) * INDEXflight(t) * INDEXpollenoutside(t)
    return(result)

### 2.7.2 Recruitment of nectar foragers

In [ ]:
def NEEDnectar(t):
    # (40)
    result = NEEDnectar_larvae(t) + NEEDnectar_adult(t)
    return(result)

In [ ]:
def NEEDnectar_larvae(t):
    # (41)
    result = 0
    for i in range(1,LIFESPANlarva):
        result += NECTARNEEDlarva(i) * LARVAE(i,t)
    return(result)

In [ ]:
def NEEDnectar_adult(t):
    # (42)
    result = BEESadult(t) * NECTARNEEDadult + NURSES(t) * NECTARNEEDnurse + FORAGESactive(t) * NECTARNEEDactiveforager
    return(result)

In [ ]:
def WORKFORCEnectar(t):
    # (43)
    result = 0
    if (RATIOworkforce(t) == 1):
        result = (BEESadult(t) * (1 - FACTORothertasks)) - NURSES(t) - FORAGERSpollen(t)
    return(result)

In [ ]:
def FORAGERSnectar(t):
    # (44)
    min1 = (BEESadult(t) * FACTORforagingmax) - FORAGERSpollen(t)
    min2 = WORKFORCEnectar(t) - PROCESSORS(t)
    result = min(min1,min2)
    return(result)

In [ ]:
def FORAGERSnectaractive(t): 
    # (45)
    result = FORAGERSnectar(t) * INDEXflight(t) * INDEXnectaroutside(t)
    return(result)

## 2.8 Modeling the resource influx into the colony

In [ ]:
def INCOMEpollen(t):
    # (46)
    result = FORAGERpollenactive(t) * LOADpollenforager * TURNSpollenforager * FACTORforagingstoch(t) * FACTORforagingsuccess
    return(result)

In [ ]:
def FACTORforagingstoch(t):
    # (47)
    if (stochastic_factor > 0.24 and stochastic_factor < 0.26):
        result = random.randint(75,125) /100
    else:
        result = 1
    return(result)

In [ ]:
def INDEXpollensituation(t):
    # (48)
    min1 = STORESpollen(t) / (NEEDpollen(t) * FACTORpollenstorage + 1)
    result = min(1,min1)
    return(result)

In [ ]:
def INCOMEnectar(t):
    # (49)
    min1 = FORAGERSnectaractive(t) * LOADnectarforager * TURNSnectarforager * FACTORforagingstoch(t) * FACTORforagingsuccess
    min2 = CELLSempty(t - 1)
    result = min(min1,min2)
    return(result)